# Line Integral Convolution

In [ ]:
import numpy as np

from scipy.ndimage import laplace

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import get_cmap

from PIL import Image

from volrender import lic

Create the grid and some vector field

In [ ]:
nx = 400
ny = 200

xi = np.linspace(0, 10, nx + 1)
yi = np.linspace(0, 5, ny + 1)

x = 0.5 * (xi[1:] + xi[:-1])
y = 0.5 * (yi[1:] + yi[:-1])

X, Y = np.meshgrid(x, y, indexing='ij')
Xi, Yi = np.meshgrid(xi, yi, indexing='ij')

vortex_x = 5.0
vortex_y = 2.5

dx = X - vortex_x
dy = Y - vortex_y

r = np.sqrt(dx**2 + dy**2)
phi = np.arctan2(dy, dx)

VX = 1     - 5 * np.exp(- r**2 / 4) * np.sin(phi)
VY = Y / 5 + 5 * np.exp(- r**2 / 4) * np.cos(phi)

# define 2D vector field and its magnitude
vel = np.array([VX, VY]).transpose(1, 2, 0)
V = np.sqrt(VX**2 + VY**2)

In [ ]:
f, ax = plt.subplots()
ax.pcolormesh(xi, yi, V.T)
s = 7
ax.quiver(x[::s], y[::s], VX.T[::s,::s], VY.T[::s,::s], color='w', pivot='mid')

# Now the LIC

Make the noise

In [ ]:
LIC = lic.LIC_twostage(x, y, vel, generate_plot=True)

Stitch LIC + Color together in HSV space

In [ ]:
rgb = lic.hsv_mix(V, LIC)

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(30, 5))
ax[0].imshow(V.T, cmap='magma', origin='lower')
ax[1].imshow(LIC.T, cmap='gray', origin='lower')
ax[2].imshow(rgb.transpose(1,0,2), origin='lower');

# Compare two approaches
one is overlaying the images via the HSV color space, the other just somehow mixes the two matrices: LIC of the vectors and magnitude

In [ ]:
f, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(8, 8))

lic.pcolormesh_rgb(x, y, rgb, ax=ax[0])
ax[0].set_title('via HSV')
ax[0].set_aspect('equal')

f = 0.25
cc=ax[1].pcolormesh(Xi, Yi, V * (1 - f + 2 * f * LIC), cmap='magma')
ax[1].set_title('matrix product')
ax[1].set_aspect('equal')